# Low Club ♣
Recreating my low club script, but better.

What we want to do is "deal" $N$ hands of Spades. We simulate the first trick as Schuster-Rules "everyone throws out their low club". We keep track of how many times each value of club wins or loses these first tricks.

How this will work in code is this:
* Make a big data frame of "hands". There are $N$ rows and 52 columns. Each column is an integer, representing a card.
    * $0$–$12$ are the 2 through Ace of Clubs
    * $13$–$25$ are the 2 through Ace of Diamonds
    * $26$–$38$ are the 2 through Ace of Hearts
    * $39$–$51$ are the 2 through Ace of Spades
* The columns come in four groups, representing the players. The values in those columns are the cards dealt to that player.
    * Columns `0`–`12` are player 1
    * Columns `13`–`25` are player 2
    * Columns `26`–`38` are player 3
    * Columns `39`–`51` are player 4
* We want to keep stats on which cards win or lose each game.
    * Look at each player's cards by breaking the columns into groups
    * Check if any player has all spades (this is an incredibly rare edge case, explained more below)
    * Find each player's lowest club. This is almost always their lowest card, unless their lowest card is above 12 in which case they have no clubs and cannot win the trick.
    * The highest low club wins the trick, and the others lose.
    

## Edge case: all spades
Spades are a trump suit, so if a spade comes out on a trick it will take the trick (unless a higher value spade plays over it). But no one can play spades until they are "broken", and they can't be broken on the first trick. *Unless* someone has a hand of 13 spades. In that case, they win all the tricks.

The odds of this happening are incredibly low. But if we play a lot of games, it is something we may see. So we have to check for it.

In [2]:
import numpy as np
import pandas as pd

N = 100

### Generate the hands
Permute the integers in $range(52)$ in $N$ rows.

In [3]:
hands = pd.DataFrame([np.random.permutation(52) for i in range(N)])
hands.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
66,28,29,45,20,21,38,26,15,19,51,...,48,4,6,37,22,10,18,24,5,12
41,27,33,31,30,28,43,19,0,50,35,...,3,39,47,32,17,13,49,5,40,1
50,0,48,5,25,8,16,10,19,6,51,...,2,11,34,13,37,23,38,24,7,31
90,4,0,32,36,20,37,33,7,31,46,...,16,24,13,10,29,40,15,14,38,1
87,36,27,11,40,51,46,49,24,16,18,...,2,7,17,10,47,32,6,1,0,22
80,41,22,32,2,36,25,24,45,29,3,...,0,15,12,13,33,46,49,1,47,19
5,5,14,33,41,0,37,26,32,39,34,...,21,15,17,27,24,11,48,8,46,6
9,34,20,0,5,28,21,27,4,35,6,...,26,25,42,9,31,39,16,12,51,7
4,47,1,17,21,36,2,18,4,42,33,...,3,12,39,48,44,45,13,51,14,40
52,32,24,34,14,31,3,20,49,51,7,...,25,43,33,16,27,6,45,10,47,19


Get the lowest card for each player and each game.

In [4]:
low_clubs = pd.concat([hands[list(range(13*i, 13*(i+1)))].apply(min, axis=1) for i in range(4)], axis=1)
low_clubs.sample(10)

,0,1,2,3
62,0,3,1,2
68,0,1,5,10
92,4,3,1,0
53,5,0,3,1
91,0,10,11,1
60,0,5,15,7
29,2,0,3,1
35,0,1,10,3
72,1,8,2,0
82,5,3,1,0


## Finding outcomes

* Find the highest club by finding the max card in each row lower than 13.
* But also check for the "all spades" case by seeing if any of the lowest cards in a row is 39.

How do I do this elegantly? I need to pull out the winning/highest low club seperately from the other losing low clubs. And I need to count them all into some stats structures.

This is good! I have the columns where the low clubs happen, and their values.

In [5]:
winning_low_club_col = low_clubs.apply(lambda row: row[(row<13) | (row==39)].idxmax(), axis=1)
winners = low_clubs.lookup(winning_low_club_col.index, winning_low_club_col.values)
winners

array([ 4,  6,  7,  4, 10,  6,  5,  3,  9,  7,  9,  6,  5,  9,  7,  6, 10,
       12, 11,  5,  5,  5,  5, 10,  3, 12,  8, 11,  6,  3,  5,  6,  4,  7,
        8, 10,  6,  4, 12,  5,  6,  8, 10,  5,  7, 10,  3,  4,  7,  5,  3,
        4, 11,  5, 12,  5,  6,  4,  6,  6,  7, 10,  3,  7, 11,  7, 11,  6,
       10,  3,  7,  5,  8,  4,  3,  4,  4,  7,  3,  9,  6,  8,  5,  3,  4,
        3,  2, 11, 10, 11,  3, 11,  4,  5,  3,  4,  9, 12,  3,  4])

Now, how to get the values from the *other* columns?

Hang on a sec. Let's think about this. I don't care about getting the losing cards separately from the winning cards. All I care about is 
* How many times did each card win?
* How many times did each card appear?

Together, these two numbers can give me the win percent. That's what I want in the end. So I don't need to try to pull out losers separately from winners. I can get winners, and I can also get everything, and that's what I need.

In [22]:
np.bincount(winners)[:13]

array([ 0,  0,  1, 14, 14, 15, 13, 11,  5,  5,  9,  8,  5])

In [23]:
np.bincount(low_clubs.values.ravel())[:13]

array([100,  81,  52,  52,  28,  22,  14,  13,   6,   5,  10,   8,   5])

Ok, let's do this for real now.

In [ ]:
def club_stats(N):
    """Generate stats for low clubs by simulating N hands
    
    Returns a data frame with rows for each card, and columns for
    - wins
    - appearances
    - win % (=wins/appearances)
    """
    